##  HW3 - Flatting JSON data from Yelp




**Submission Instruction**

1- Replace black in the tile with your name.

2- Complete your notebook and run all the cells

3- Download .ipynb

4- Submit .ipynb file on Gradescope

**Setting Up Yelp API**

For this assignment we're going to be using the Yelp API via the python package `yelpapi`.  Like with the Spotify API, you need to go get a yelp developer account here: https://www.yelp.com/fusion. Here is step-by-step guide:

1- Go to https://www.yelp.com/fusion

2- Click on Start Free Trial and Click on Start Free 30-day Trial on Premium

3- Sign Up and Confirm your email address

4- After Signing Up you will see the Create New App page.

5- Put a name for your app, you can select Education as Industry. As your contact email put it a made up business email. I used riazi@microsoft.com. Finally select Premium as the Access Tier (without Premium access you cannot fetch reviews from Yelp and cannot complete this notebook). You can put NA under description.

6- Collect the write down your API Key. When you get the API key you can proceed with the homework.

The goal will be to make a dataset of local taco places that are doing well during Coronavirus. Specifically, we want to make a dataset that takes their average score since they've been open and compares it to the average score of the last three reviews.  Shops that have been doing well should hopefully have these two averages be similar.

There will be three main steps to this process:

* First you'll search by location type and get aggregate information for everything that falls in the ice cream category.
* Second you'll get reviews for all those locations.  Yelp only returns three reviews when you call an ID, but that still works. Given you can only query one ID at a time, you'll need to write a loop to create a dataframe of all the reviews.  
* Third, you'll aggregate the latest review information to see how their average review score compares to their overall average review score.  

In [ ]:
# Run this cell the first time, but after installing the yelpapi you need to reset the session. After reset skip this cell
# Mount google drive
import os, sys
from google.colab import drive
drive.mount('/content/mnt')
nb_path = '/content/notebooks'
os.symlink('/content/mnt/My Drive/Colab Notebooks', nb_path)
sys.path.insert(0, nb_path)  # or append(nb_path)

Mounted at /content/mnt


In [4]:
#Run this cell after the resetting the session
# Mount google drive
import os, sys
from google.colab import drive
drive.mount('/content/mnt')
nb_path = '/content/notebooks'

sys.path.insert(0, nb_path)  # or append(nb_path)

Drive already mounted at /content/mnt; to attempt to forcibly remount, call drive.mount("/content/mnt", force_remount=True).


In [ ]:
# Install yelpapi once. Next time you run this notebook, you can skip this.
!pip install --target=$nb_path yelpapi

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 161.2/161.2 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 151.8/151.8 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.4/70.4 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.8/129.8 kB 10.1 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.


In [5]:
# Now enter your API key so you can make requests
from yelpapi import YelpAPI
from pandas import json_normalize
# yelp_api = YelpAPI('ENTER YOUR KEY HERE')
api_key = 'sO5J95emrHDhjLaejpb5gSs5VY-jnc8C7AvZBKrn_I4TTQ1wQr1NdOB0jzcieu2YJxU7gQnbLB8Sv6LYV_wv-EASD-J1nBnB5cwAN-RFz1_Ou8qDz_5-dYaCi2HMaHYx'
yelp_api = YelpAPI(api_key)


### Making calls to with Yelp API

There are many functions in the `yelpapi` package.  The first one we'll use is `search_query()`.  You can put in a term you want to search for followed up by the location and it'll give you all the locations that match.  For example, the following would search for all ice cream places here in Chicago.

```
ice = yelp_api.search_query(term = 'ice cream', location = 'Chicago, IL')
```

This will give you a response of all the ice cream places in Chicago. It'll be in JSON form so will need flattening before being useful.

### Getting all taco shops and flattening - [6 points]

**Task:** Start by making a dataframe that uses the `search_query()` function to search using the term 'taco'.  Call this `taco_shops`.  After that, flatten the json results to `taco_shops_df`.

In [7]:
# search for taco shops and store to taco_shops
taco_shops = yelp_api.search_query(term = 'taco',  location='Chicago, IL')

In [8]:
# Look at taco_shops
taco_shops

{'businesses': [{'id': 'CADdsveMbJY-x_PFrsCkQw',
   'alias': 'el-tragon-taqueria-chicago-5',
   'name': 'El Tragon Taqueria',
   'image_url': 'https://s3-media0.fl.yelpcdn.com/bphoto/EANG1s0BojybA2qEx0w7Iw/o.jpg',
   'is_closed': False,
   'url': 'https://www.yelp.com/biz/el-tragon-taqueria-chicago-5?adjust_creative=LkXibxc44XKkEcnrS_SRfA&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=LkXibxc44XKkEcnrS_SRfA',
   'review_count': 156,
   'categories': [{'alias': 'tacos', 'title': 'Tacos'}],
   'rating': 4.7,
   'coordinates': {'latitude': 41.904700405779366,
    'longitude': -87.64861798544506},
   'transactions': ['delivery', 'pickup'],
   'location': {'address1': '1234 N Halsted St',
    'address2': 'Unit C',
    'address3': '',
    'city': 'Chicago',
    'zip_code': '60642',
    'country': 'US',
    'state': 'IL',
    'display_address': ['1234 N Halsted St', 'Unit C', 'Chicago, IL 60642']},
   'phone': '+13123741458',
   'display_phone': '(312) 374-1458',
   'di

In [9]:
# What keys are present?
taco_shops.keys()

dict_keys(['businesses', 'total', 'region'])

Now use the json_normalize function to flatten `taco_shops`.  Note that you need to select the key that contains the businesses when flattening.  But this one is easier than the examples from the homework in that you don't need to provide any other arguments.  

Store the result as `taco_shops_df`

After that select only the columns 'id', 'alias', 'name', 'review_count', and 'rating'.

In [11]:
# Flatten to taco_shops_df
taco_shops_df = json_normalize(taco_shops, record_path='businesses')
taco_shops_df.head() # Check

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,transactions,...,attributes.has_pool_table,attributes.restaurants_attire,attributes.best_nights.monday,attributes.best_nights.tuesday,attributes.best_nights.friday,attributes.best_nights.wednesday,attributes.best_nights.thursday,attributes.best_nights.sunday,attributes.best_nights.saturday,attributes.service_locations_biz_dict
0,CADdsveMbJY-x_PFrsCkQw,el-tragon-taqueria-chicago-5,El Tragon Taqueria,https://s3-media0.fl.yelpcdn.com/bphoto/EANG1s...,False,https://www.yelp.com/biz/el-tragon-taqueria-ch...,156,"[{'alias': 'tacos', 'title': 'Tacos'}]",4.7,"[delivery, pickup]",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,HiCXpGgxqjHPIlLyoZ06yw,carniceria-maribel-chicago,Carniceria Maribel,https://s3-media0.fl.yelpcdn.com/bphoto/qbrsZA...,False,https://www.yelp.com/biz/carniceria-maribel-ch...,170,"[{'alias': 'beer_and_wine', 'title': 'Beer, Wi...",4.8,[delivery],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,-HqkUZg3tu-ZAV1RhdzXHQ,tacotlan-chicago,Tacotlan,https://s3-media0.fl.yelpcdn.com/bphoto/7bAgkw...,False,https://www.yelp.com/biz/tacotlan-chicago?adju...,423,"[{'alias': 'mexican', 'title': 'Mexican'}, {'a...",4.3,"[delivery, pickup]",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,d9LQsJDCgm3_hVu7ePVyQA,l-patron-chicago-5,L' Patron,https://s3-media0.fl.yelpcdn.com/bphoto/MjXhkE...,False,https://www.yelp.com/biz/l-patron-chicago-5?ad...,20,"[{'alias': 'tacos', 'title': 'Tacos'}]",4.0,[],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,VNTJYUiOj6hD2Uac_prtPQ,taqueria-chingon-chicago-2,Taqueria Chingon,https://s3-media0.fl.yelpcdn.com/bphoto/UgGmI8...,False,https://www.yelp.com/biz/taqueria-chingon-chic...,13,"[{'alias': 'tacos', 'title': 'Tacos'}]",4.2,"[delivery, pickup]",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
# Select only necessary columns
taco_shops_df = taco_shops_df[['id', 'alias', 'name', 'review_count', 'rating']]
taco_shops_df.shape # Check shape.  Is it 20 x 5?

(20, 5)

## Getting reviews for the taco shops - [12 points]

Now we're going to use the `reviews_query()` function to get the last three reviews for a given ID.  The issue here is that you can only feed it one ID at a time.  So, we'll have to write a loop that queries for each ID in `taco_shops_df` and builds out a dataframe of reviews.  

**Task:**  Write a for loop that does the following steps:
* First make an empty list outside of the loop called `taco_shops_reviews_df`
* Initalize your loop so that it runs the length of `taco_shops_df` you made earlier.
* For each i in loop, use the id from `taco_shops_df` to get reviews from yelp using the `reviews_query()` function in `yelp_api` and store it to an object called `reviews`.
* Flatten `reviews` to an objected called `reviews_df`
* Add `location_id` to `reviews_df` - I gave you the code to do this :)
* Append `reviews_df` to `taco_shops_reviews_df`
* After the loop Transform `taco_shops_reviews_df` to a dataframe with the same name using concat
* Finally select only the columns `'id', 'text', 'rating', 'time_created', 'location_id'`
* Note: You need to slow down retreving the reviews using time.sleep(1)

In [19]:
import time
import pandas as pd
# Write our loop
taco_shops_reviews_df = list()

for i in range(len(taco_shops_df)):
  reviews = yelp_api.reviews_query(taco_shops_df['id'][i])
  reviews_df = json_normalize(reviews, record_path='reviews')
  reviews_df['location_id'] = taco_shops_df['id'][i]
  taco_shops_reviews_df.append(reviews_df)
  time.sleep(1)

taco_shops_reviews_df = pd.concat(taco_shops_reviews_df)


In [22]:
# Select columns 'id', 'text', 'rating', 'time_created', 'location_id'
taco_shops_reviews_df = taco_shops_reviews_df[['id','text','rating','time_created','location_id']]
taco_shops_reviews_df

,id,text,rating,time_created,location_id
0,rn07QpC-IRqFcgU5T53qRg,Food: Amazing!! Very authentic and has a lot o...,5,2025-07-07 20:31:42,CADdsveMbJY-x_PFrsCkQw
1,Ln_ijWrj3foODnOlSpgd5A,For the best tacos in Chicago - look no furthe...,5,2025-06-03 21:34:53,CADdsveMbJY-x_PFrsCkQw
2,tY1juHxL7yJ0fu9MMjGVdA,"Best tacos in Chicago, no competition! Truly a...",5,2025-06-03 05:43:10,CADdsveMbJY-x_PFrsCkQw
3,ND_Y-44EOiqGCHBqVNWX3Q,"Very VERY good tacos, fresh, super authentic a...",5,2025-09-03 10:19:28,CADdsveMbJY-x_PFrsCkQw
4,b6bUEh949_g2yFr1qqXDog,After hearing great things about El Tragon Taq...,4,2025-03-07 19:42:39,CADdsveMbJY-x_PFrsCkQw
...,...,...,...,...,...
2,3ybIE-BmpKVHR0sr35D40g,The best prices for all your meat and vegetabl...,5,2025-07-16 09:56:50,jbCkRZUpzXcG1PYbgiby4g
3,h7Rk4xoxN7mywCSnyvbT9Q,Love this taco joint tucked inside the grocery...,5,2025-04-13 11:57:54,jbCkRZUpzXcG1PYbgiby4g
4,NFKu2zIInZMYak44o66IFQ,Overall Thoughts: Carnicerias Guanajuato is a ...,4,2024-01-14 14:11:09,jbCkRZUpzXcG1PYbgiby4g
5,uxI6lyxuSmqVnMHhrrOfgQ,Carnicerias Guanajuato is a hidden gem that ne...,5,2024-06-23 16:18:58,jbCkRZUpzXcG1PYbgiby4g


### Aggregating your review data - [6 points]

**Task:** Now go and do a data aggregation to get the mean review score across the three reviews. Remember, we want this grouped by location_id.  Call this dataframe `latest_reviews_agg`.

In your groupby you should set `as_index` to false to make joining on the ID values easier.

You should also rename the second column to `rating_mean` vs. leaving it as the dual level name.


In [23]:
# Do your groupby to get mean rating.
# Call it latest_reviews_agg.
latest_reviews_agg = taco_shops_reviews_df.groupby('location_id', as_index=False)['rating'].mean()

In [24]:
# Check it!
latest_reviews_agg

,location_id,rating
0,-HqkUZg3tu-ZAV1RhdzXHQ,4.714286
1,0Zu5S3zJvLC1Ve7kGAttDQ,4.285714
2,BGTumAF6UxyE-TrEB11WVg,4.000000
3,CADdsveMbJY-x_PFrsCkQw,4.857143
4,Gr42JZ9GsKaESW-864rwFQ,5.000000
5,HiCXpGgxqjHPIlLyoZ06yw,4.571429
6,RIEypHKiSCZB5mJJJQNMcQ,4.714286
7,RI_TzjULvOknWecEq8cBQA,5.000000
8,VNTJYUiOj6hD2Uac_prtPQ,4.857143
9,Yzs5dkjw3hEkM00RgBwNag,4.857143


In [27]:
# Rename so the two column names are 'location_id' and 'mean_rating'
latest_reviews_agg = latest_reviews_agg.rename(columns={'rating':'mean_rating'})

### Join your two datasets and one last transform - [6 points]

**Task:** Now it's time to join `latest_reviews_agg` back to your `taco_shops_df` dataframe.  You're going to want to join them on the location_id, but remember it is called just 'id' in the `taco_shops_df`  dataframe.  Call this joined dataset `taco_shops_comp`

After you make that dataset do one last transform.  In this I want you to make a new column called 'still_good' where the value is 'yes' if the mean_rating is greater than or equal to the average rating since they opened, or 'no' if the rating has dropped.   The idea here is that this could be a value that one would use to see if their average score of the latest reviews has improved or suffered during Covid.  

In [29]:
# Join and name taco_shops_comp

taco_shops_comp = taco_shops_df.merge(latest_reviews_agg, left_on='id', right_on='location_id')
taco_shops_comp

,id,alias,name,review_count,rating,location_id,mean_rating
0,CADdsveMbJY-x_PFrsCkQw,el-tragon-taqueria-chicago-5,El Tragon Taqueria,156,4.7,CADdsveMbJY-x_PFrsCkQw,4.857143
1,HiCXpGgxqjHPIlLyoZ06yw,carniceria-maribel-chicago,Carniceria Maribel,170,4.8,HiCXpGgxqjHPIlLyoZ06yw,4.571429
2,-HqkUZg3tu-ZAV1RhdzXHQ,tacotlan-chicago,Tacotlan,423,4.3,-HqkUZg3tu-ZAV1RhdzXHQ,4.714286
3,d9LQsJDCgm3_hVu7ePVyQA,l-patron-chicago-5,L' Patron,20,4.0,d9LQsJDCgm3_hVu7ePVyQA,4.428571
4,VNTJYUiOj6hD2Uac_prtPQ,taqueria-chingon-chicago-2,Taqueria Chingon,13,4.2,VNTJYUiOj6hD2Uac_prtPQ,4.857143
5,oxJFCIVL9oQcmSIVSOBYew,taco-el-jalisciense-chicago-3,Taco El Jalisciense,206,4.3,oxJFCIVL9oQcmSIVSOBYew,4.714286
6,BGTumAF6UxyE-TrEB11WVg,antique-taco-chicago-11,Antique Taco,1400,4.1,BGTumAF6UxyE-TrEB11WVg,4.000000
7,RIEypHKiSCZB5mJJJQNMcQ,birrieria-zaragoza-chicago-2,Birrieria Zaragoza,106,4.6,RIEypHKiSCZB5mJJJQNMcQ,4.714286
8,0Zu5S3zJvLC1Ve7kGAttDQ,bárbaro-taquería-chicago,Bárbaro Taquería,206,4.5,0Zu5S3zJvLC1Ve7kGAttDQ,4.285714
9,hDpTtEEuYq5eez-ppNddgA,tts-tacos-and-tequilas-chicago,TTS Tacos & Tequilas,34,4.7,hDpTtEEuYq5eez-ppNddgA,4.285714


When you are comparing the columns make sure to not comparing taco_shops_df and taco_shops_comp since their tuples may have different orders! Use both columns from taco_shops_comp

In [31]:
import numpy as np
# Make still_good column
taco_shops_comp['still_good'] = np.where(taco_shops_comp['mean_rating'] >= taco_shops_comp['rating'], "Yes", "No")

In [34]:
# So, do any locations have a lower average in their last three reviews compared to their average overall score?
#Find the name of taco shops are not good anymore.
print(taco_shops_comp[taco_shops_comp['still_good'] == "No"]['name'])

1        Carniceria Maribel
6             Antique Taco 
8          Bárbaro Taquería
9      TTS Tacos & Tequilas
13        Onions & Cilantro
14                Taco City
15    Torteria San Lenchito
18    Don Pez Tacos Cantina
Name: name, dtype: object
